In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\Swapnil Jyot\.conda\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import langchain
import pinecone

print("LangChain version:", langchain.__version__)
print("Pinecone client version:", pinecone.__version__)


LangChain version: 0.2.5
Pinecone client version: 4.1.1


In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

c:\Users\Swapnil Jyot\.conda\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [11]:
import os
os.environ["PINECONE_API_KEY"] = "aa0c6b5f-16cf-4fd9-8b32-e1ad555a3382"

In [12]:
import os
from pinecone import Pinecone, ServerlessSpec

In [13]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [14]:
pc

In [15]:
pc.list_indexes().names()

['medical-chatbot', 'fluxdatabase']

In [17]:
index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [18]:
print(index_name)

medical-chatbot


In [19]:
from langchain.vectorstores import Pinecone as LangchainPinecone

In [20]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import OpenAIEmbeddings  # or the embedding model you are using
import pinecone

In [21]:
# Create the document search index
docsearch = LangchainPinecone.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name
)

In [22]:
docsearch #this is a vector database

In [23]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x00000277D2221EB0>)

In [24]:
query = "What are Allergies"

In [25]:
docs=docsearch.similarity_search(query, k=3)

In [26]:
print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which co

In [27]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [28]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [30]:
import requests
import os

# Create the directory if it doesn't exist
model_directory = r"C:\Users\Swapnil Jyot\AI_MEDICAL_CHATBOT\model"

os.makedirs(model_directory, exist_ok=True)

# Hugging Face repository and file path
repo_id = "TheBloke/Llama-2-7B-Chat-GGML"
filename = "llama-2-7b-chat.ggmlv3.q2_K.bin"
download_url = f"https://huggingface.co/{repo_id}/resolve/main/{filename}"

# Download the model file from Hugging Face
model_path = os.path.join(model_directory, filename)

# Make a request with streaming enabled
response = requests.get(download_url, stream=True)

# Get the total file size from the headers
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # Size of each block in bytes

# Initialize the progress bar
from tqdm import tqdm

tqdm_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Download the file in chunks
with open(model_path, 'wb') as f:
    for data in response.iter_content(block_size):
        tqdm_bar.update(len(data))
        f.write(data)

tqdm_bar.close()

# Print the path where the model is saved
print(f"Model downloaded to: {model_path}")

100%|██████████| 2.87G/2.87G [17:16<00:00, 2.77MiB/s]  


Model downloaded to: C:\Users\Swapnil Jyot\AI_MEDICAL_CHATBOT\model\llama-2-7b-chat.ggmlv3.q2_K.bin


In [54]:

llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':100,
                          'temperature':0.8})
                    

In [56]:
# Assuming 'model_path' is defined from previous code
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 100,  # Lowered from 512 for faster response
        'temperature': 0.7,  # Slightly reduced temperature
        'top_p': 0.9,
        'repetition_penalty': 1.0
    }
)

In [57]:
# Assuming `llm` is the loaded model instance from ctransformers

# Example query or prompt
query = "Who is Mahatma Gandhi"

# Parameters for text generation
params = {
    "max_new_tokens": 50,  # Use 'max_new_tokens' to control the length
    "temperature": 0.9,
    "top_p": 0.9,
    "repetition_penalty": 1.0
}

# Generate text based on the query
outputs = llm(query, **params)  # Pass parameters as a dictionary

# Print the generated outputs
print(outputs)

c:\Users\Swapnil Jyot\.conda\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [1]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'RetrievalQA' is not defined

In [31]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])